In [51]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_squared_error
from sklearn.feature_extraction import DictVectorizer

In [4]:
!pip install fastparquet

     |████████████████████████████████| 775 kB 1.0 MB/s eta 0:00:01
     |████████████████████████████████| 2.7 MB 2.7 MB/s eta 0:00:01


In [6]:
df_jan= pd.read_parquet('/Users/m.shark/Downloads/fhv_tripdata_2021-01.parquet', engine = 'fastparquet')

In [69]:
df_feb = pd.read_parquet('/Users/m.shark/Downloads/fhv_tripdata_2021-02.parquet', engine = 'fastparquet')

In [8]:
# 1 
len(df_jan)

1154112

In [12]:
df_jan['duration'] = df_jan['dropOff_datetime'] - df_jan['pickup_datetime']
df_jan['duration'] = df_jan['duration'].dt.total_seconds() / 60

In [70]:
df_feb['duration'] = df_feb['dropOff_datetime'] - df_feb['pickup_datetime']
df_feb['duration'] = df_feb['duration'].dt.total_seconds() / 60

In [21]:
# 2 
df_jan['duration'].mean()

19.1672240937939

In [23]:
#2.2
cond = (df_jan.duration >= 1) & (df_jan.duration <= 60)
len(df_jan[~cond])

44286

In [24]:
df_jan = df_jan[cond]

In [66]:
df_jan['PUlocationID'] = df_jan.PUlocationID.fillna(-1)
# df_feb['PUlocationID'] = df_feb.PUlocationID.fillna(-1)

In [36]:
#3
df_jan[df_jan['PUlocationID'] == -1].count()['PUlocationID'] / len(df_jan)

0.8352732770722617

In [45]:
categorical = ['PUlocationID', 'DOlocationID']
df_jan[categorical] = df_jan[categorical].astype(str)

In [46]:
train_dicts = df_jan[categorical].to_dict(orient='records')
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [48]:
# 4
X_train.shape[1]

525

In [52]:
target = 'duration'
y_train = df_jan[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_train)
mean_squared_error(y_train, y_pred, squared=False)

10.52851910721175

In [77]:
cond = (df_feb.duration >= 1) & (df_feb.duration <= 60)
df_feb = df_feb.loc[cond]
df_feb[categorical] = df_feb[categorical].astype(str)
test_dicts = df_feb[categorical].to_dict(orient='records')
X_test = dv.transform(test_dicts)

In [73]:
y_test = df_feb.duration.values

In [74]:
y_pred_test = lr.predict(X_test)
mean_squared_error(y_test, y_pred_test, squared=False)

11.371874821992126